In [1]:
import pandas as pd

Загружаем датасет: Изучение настроений покупателей в McDonald's Отзывы о магазине в США
https://www.kaggle.com/datasets/nelgiriyewithana/mcdonalds-store-reviews
Этот набор данных содержит более 33 000 анонимных отзывов о магазинах McDonald's в Соединенных Штатах, взятых из обзоров Google. Он предоставляет ценную информацию об опыте клиентов и мнениях о различных местах расположения McDonald's по всей стране. Набор данных включает такую информацию, как названия магазинов, категории, адреса, географические координаты, рейтинги отзывов, тексты отзывов и временные метки.

In [9]:
data = pd.read_csv('McDonald_s_Reviews.csv', on_bad_lines='skip', encoding='latin-1')

In [10]:
data.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [24]:
hand_marked_data = data.sample(frac=0.01)  # для ручной разметки файл

In [25]:
hand_marked_data.to_csv('review.csv', index=False)

Создаем метки для ручной разметки в LabelStudio
![Создаем метки для ручной разметки в LabelStudio](img1.png)

Ручная разметка
![Создаем метки для ручной разметки в LabelStudio](img2.png)

Затем экспортируем файл из Label Studio

Создаем функцию разметки текста на основе содержания определенных слов

In [26]:
def rule_based_labeling(text):
    positive_keywords = ['good', 'great', 'excellent', 'positive', 'joy', 'happy', 'yammy', 'tasty', 'delicious', 'fresh', 'cleanly']
    negative_keywords = ['bad', 'terrible', 'disappointed', 'negative', 'problem', 'sad', 'dirty', 'burnt', 'burn', 'old', 'dry', 'stale']

    text = text.lower()
    if any(keyword in text for keyword in positive_keywords):
        return 'positive'
    elif any(keyword in text for keyword in negative_keywords):
        return 'negative'
    else:
        return 'neutral'

Применяем разметку на основе правил к датасету

In [27]:
# marked_data = data.sample(frac=0.2)  # взятие 20% от всего датасета
marked_data = data  # ко всему блоку применяем автоматическую разметку
marked_data['sentiment'] = marked_data['review'].apply(rule_based_labeling)

In [28]:
marked_data.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating,sentiment
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,neutral
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars,neutral
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star,negative
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars,neutral
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star,neutral


In [29]:
manually_labeled_data = pd.read_csv('review_after_ls.csv')

# соединение автоматически размеченных данных с ручной разметкой
combined_data = pd.concat([marked_data, manually_labeled_data])

In [30]:
combined_data['sentiment'].fillna('neutral', inplace=True)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split  # основная задача разделить данные на две части: одна для обучения модели, другая дла проверки эффективности

x = combined_data['review']
y = combined_data['sentiment']


vectorizer = TfidfVectorizer()
x_vectorized = vectorizer.fit_transform(x)

# Разделяем данные на обучающую и тестовую выборку
x_train, x_test, y_train, y_test = train_test_split(x_vectorized, y, test_size=0.2, random_state=42)

# создание и обучение модели логистической регрессии
model = LogisticRegression()
model.fit(x_train, y_train)


C:\Users\akste\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
from sklearn.metrics import accuracy_score

# Оценка модели
# Прогнозирование меток для тестовых данных:
y_test_predicted = model.predict(x_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_test_predicted)
print(f'Точность модели: {accuracy}')

Точность модели: 0.9523026315789473


Точность модели составила 95%